In [2]:
import pandas as pd

In [3]:
def verify_snake_case(df):
    """Method to verify that the names of the dataframe columns follow the snake_case convention"""
    return [df.columns[col].replace(' ', '_') for col in range(len(df.columns))]

In [4]:
def verify_schema(df):
    """Verifies the columns of dataframe 'df' are consistent with those of the call center schema.
    If schema is not correct, returns missing/additional columns"""

    # Remove unnamed columns
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    
    # Columns of call center schema
    cols = {'prequeue', 'id_cmpg', 'agn_name', 'cont_phone', 'sent_name', 'abandon', 'contact_id', 'cont_last_name',
            'cont_email', 'skill_sentiment', 'date', 'start_date', 'chan_priority', 'total_time', 'agent_time',
            'skill_disp_code', 'chan_type', 'master_id', 'skill_importance', 'start_time', 'id_cont', 'chan_name',
            'cmpg_name', 'inqueue', 'sla', 'id_skill', 'agn_email', 'ccl_name', 'postqueue', 'id_rsn', 'rsn_name',
            'id_chan', 'skill_name', 'id_ccl', 'id_agn', 'ccl_country', 'id_sent', 'cont_name', 'team_name',
            'last_name', 'id_team', 'abandon_time'}

    if cols == set(df.columns):
        return True
    else:
        # return missing and additional columns
        return list(sorted(set(df.columns) - cols)), list(sorted(cols - set(df.columns)))

In [5]:
def set_datatype(df):
    """Method to set datatypes of dataframe consistent with call center schema.
    If schema is not correct, returns missing/additional columns"""

    if verify_schema(df) is True:
        df['start_date'] = pd.to_datetime(df['start_date'])
        df['abandon'] = df['abandon'].astype(int)
        df['prequeue'] = df['prequeue'].astype(int)
        df['inqueue'] = df['inqueue'].astype(int)
        df['agent_time'] = df['agent_time'].astype(int)
        df['postqueue'] = df['postqueue'].astype(int)
        df['total_time'] = df['total_time'].astype(int)
        df['sla'] = df['sla'].astype(pd.Int64Dtype())
        df['contact_id'] = df['contact_id'].astype(str)
        df['master_id'] = df['master_id'].astype(str)
        df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
        df['start_time'] = pd.to_datetime(df['start_time'], infer_datetime_format=True)
        df['abandon_time'] = df['abandon_time'].astype(int)
        df['id_agn'] = df['id_agn'].astype(str)
        df['id_ccl'] = df['id_ccl'].astype(str)
        df['id_cont'] = df['id_cont'].astype(str)
        df['id_cmpg'] = df['id_cmpg'].astype(str)
        df['id_sent'] = df['id_sent'].astype(str)
        df['id_chan'] = df['id_chan'].astype(str)
        df['id_rsn'] = df['id_rsn'].astype(str)
        df['agn_name'] = df['agn_name'].astype(str)
        df['last_name'] = df['last_name'].astype(str)
        df['agn_email'] = df['agn_email'].astype(str)
        df['id_team'] = df['id_team'].astype(int)
        df['id_skill'] = df['id_skill'].astype(int)
        df['skill_name'] = df['skill_name'].astype(str)
        df['skill_disp_code'] = df['skill_disp_code'].astype(int)
        df['skill_importance'] = df['skill_importance'].astype(int)
        df['skill_sentiment'] = df['skill_sentiment'].astype(str)
        df['ccl_name'] = df['ccl_name'].astype(str)
        df['ccl_country'] = df['ccl_country'].astype(str)
        df['cont_name'] = df['cont_name'].astype(str)
        df['cont_last_name'] = df['cont_last_name'].astype(str)
        df['cont_email'] = df['cont_email'].astype(str)
        df['cont_phone'] = df['cont_phone'].astype(str)
        df['cmpg_name'] = df['cmpg_name'].astype(str)
        df['sent_name'] = df['sent_name'].astype(str)
        df['chan_type'] = df['chan_type'].astype(str)
        df['chan_name'] = df['chan_name'].astype(str)
        df['chan_priority'] = df['chan_priority'].astype(int)
        df['rsn_name'] = df['rsn_name'].astype(str)

        # Remove unnamed columns
        return df.loc[:, ~df.columns.str.contains('^Unnamed')]
    else:
        return 'Missing columns: {}\n' \
               'Additional columns: {}'.format(verify_schema(df)[0], verify_schema(df)[1])

In [6]:
def load_call_logs(file, flat=False, sheet='call_logs'):
    """A method to load call_logs files of extension txt, cvs, xlsx, parquet or json.
    Resulting dataframe has columns that follow snake case convention. Encoding of text is set as UTF-8
        file: name of file with path included
        flat: if True, it flattens the dataframe. Default is False
        sheet: in case it is a csv or xlsx file, it specifies the sheet name. Default is 'call_logs'"""

    # Open file
    if file.endswith('.txt'):
        df = pd.read_csv(file, delimiter=',', encoding='utf8')
    elif file.endswith('.csv'):
        df = pd.read_csv(file, encoding='utf8')
    elif file.endswith('.xlsx'):
        df = pd.read_excel(file, sheet_name=sheet)
    elif file.endswith('.parquet'):
        df = pd.read_parquet(file)
    elif file.endswith('.json'):
        df = pd.read_json(file, encoding='utf8')
    else:
        return 'Not a valid extension'

    # Set datatype according to call center schema
    df = set_datatype(df)

    if isinstance(df, pd.DataFrame):

        # Set columns following snake case convention
        df.set_axis(verify_snake_case(df), axis=1, inplace=True)

        # Remove unnamed columns
        df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

        # Flatten dataframe is flat=True
        if flat is True:
            return df.to_numpy().flatten()
        else:
            return df
    else:
        return df


In [8]:
test = load_call_logs('../data/raw/05-call_logs.txt')
print(test)

               start_date  abandon  prequeue  inqueue  agent_time  postqueue  \
0     2019-05-01 22:07:00        1         6      138           0          0   
1     2019-05-31 10:02:00        0         5       53           8          0   
2     2019-05-04 16:10:00        0         9        4          27          0   
3     2019-05-24 13:31:00        0         5       20          28          0   
4     2019-05-19 17:24:00        1         0        0           0          0   
...                   ...      ...       ...      ...         ...        ...   
39548 2019-05-11 10:07:00        0         6       49          73          0   
39549 2019-05-12 10:27:00        0         6      137          56          0   
39550 2019-05-09 16:19:00        0         6       11         132          0   
39551 2019-05-10 08:56:00        0         6      224           2          0   
39552 2019-05-24 19:18:00        0        12        0         152          0   

       total_time   sla    contact_id  

In [9]:
test2 = load_call_logs('../data/raw/03-call_logs.parquet')
print(test2)

               start_date  abandon  prequeue  inqueue  agent_time  postqueue  \
0     2019-03-01 00:14:00        1        15        0           0          0   
1     2019-03-10 07:55:00        0         6      160         290          0   
2     2019-03-23 09:15:00        0         5      176          65          0   
3     2019-03-11 10:22:00        0         6      408         473          0   
4     2019-03-15 15:38:00        0         6       85           2          0   
...                   ...      ...       ...      ...         ...        ...   
38796 2019-03-11 15:47:00        1        10       72           0          0   
38797 2019-03-18 16:54:00        1        10      432           0          0   
38798 2019-03-12 09:04:00        1         6       14           0          0   
38799 2019-03-28 09:07:00        1         5       22           0          0   
38800 2019-03-03 10:54:00        1         6       11           0          0   

       total_time   sla    contact_id  